In [1]:
from pyspark.sql import SQLContext
from pyspark.sql.types import *
from pyspark.sql import functions
sqlContext = SQLContext
print(sc)

In [2]:
schema = StructType([
        StructField("GLOBALEVENTID", StringType(), True),
        StructField("SQLDATE", IntegerType(), True),
        StructField("MonthYear", IntegerType(), True),
        StructField("Year", IntegerType(), True),
        StructField("FractionDate", FloatType(), True),
        StructField("Actor1Code", StringType(), True),
        StructField("Actor1Name", StringType(), True),
        StructField("Actor1CountryCode", StringType(), True),
        StructField("Actor1KnownGroupCode", StringType(), True),
        StructField("Actor1EthnicCoe", StringType(), True),
        StructField("Actor1Religion1Code", StringType(), True),
        StructField("Actor1Religion2Code", StringType(), True),
        StructField("Actor1Type1Code", StringType(), True),
        StructField("Actor1Type2Code", StringType(), True),
        StructField("Actor1Type3Code", StringType(), True),
        StructField("Actor2Code", StringType(), True),
        StructField("Actor2Name", StringType(), True),
        StructField("Actor2CountryCode", StringType(), True),        
        StructField("Actor2KnownGroupCode", StringType(), True),
        StructField("Actor2EthnicCoe", StringType(), True),
        StructField("Actor2Religion1Code", StringType(), True),
        StructField("Actor2Religion2Code", StringType(), True),
        StructField("Actor2Type1Code", StringType(), True),
        StructField("Actor2Type2Code", StringType(), True),
        StructField("Actor2Type3Code", StringType(), True),
        StructField("IsRootEvent", StringType(), True),
        StructField("EventCode", StringType(), True),
        StructField("EventBaseCode", StringType(), True),
        StructField("EventRootCode", StringType(), True),
        StructField("QuadClass", StringType(), True),
        StructField("GoldsteinScale", FloatType(), True),
        StructField("NumMentions", IntegerType(), True),
        StructField("NumSources", IntegerType(), True),
        StructField("NumArticles", IntegerType(), True),
        StructField("AvgTone", FloatType(), True),
        StructField("Actor1Geo_Type", StringType(), True),
        StructField("Actor1Geo_FullName", StringType(), True),
        StructField("Actor1Geo_CountryCode", StringType(), True),
        StructField("Actor1Geo_ADM1Code", StringType(), True),
        StructField("Actor1Geo_Lat", FloatType(), True),
        StructField("Actor1Geo_Long", FloatType(), True),
        StructField("Actor1Geo_FeatureIDE", StringType(), True),
        StructField("Actor2Geo_Type", StringType(), True),
        StructField("Actor2Geo_FullName", StringType(), True),
        StructField("Actor2Geo_CountryCode", StringType(), True),
        StructField("Actor2Geo_ADM1Code", StringType(), True),
        StructField("Actor2Geo_Lat", FloatType(), True),
        StructField("Actor2Geo_Long", FloatType(), True),
        StructField("Actor2Geo_FeatureIDE", StringType(), True),
        StructField("ActionGeo_Type", StringType(), True),
        StructField("ActionGeo_FullName", StringType(), True),
        StructField("ActionGeo_CountryCode", StringType(), True),
        StructField("ActionGeo_ADM1Code", StringType(), True),
        StructField("ActionGeo_Lat", FloatType(), True),
        StructField("ActionGeo_Long", FloatType(), True),
        StructField("ActionGeoo_FeatureIDE", StringType(), True),
        StructField("DATEADDED", IntegerType(), True),
        StructField("SOURCEURL", StringType(), True),
    ])

raw = spark.read.csv("file:///home/javi/masterdatascience/TFM/pruebas/gdelt/unzipped/*", header=False, schema = schema, sep = '\t')
raw

# Es importante que en la ruta unicamente esten los archivos .csv

DataFrame[GLOBALEVENTID: string, SQLDATE: int, MonthYear: int, Year: int, FractionDate: float, Actor1Code: string, Actor1Name: string, Actor1CountryCode: string, Actor1KnownGroupCode: string, Actor1EthnicCoe: string, Actor1Religion1Code: string, Actor1Religion2Code: string, Actor1Type1Code: string, Actor1Type2Code: string, Actor1Type3Code: string, Actor2Code: string, Actor2Name: string, Actor2CountryCode: string, Actor2KnownGroupCode: string, Actor2EthnicCoe: string, Actor2Religion1Code: string, Actor2Religion2Code: string, Actor2Type1Code: string, Actor2Type2Code: string, Actor2Type3Code: string, IsRootEvent: string, EventCode: string, EventBaseCode: string, EventRootCode: string, QuadClass: string, GoldsteinScale: float, NumMentions: int, NumSources: int, NumArticles: int, AvgTone: float, Actor1Geo_Type: string, Actor1Geo_FullName: string, Actor1Geo_CountryCode: string, Actor1Geo_ADM1Code: string, Actor1Geo_Lat: float, Actor1Geo_Long: float, Actor1Geo_FeatureIDE: string, Actor2Geo_Ty

In [8]:
data = raw.select('Actor1CountryCode','Actor2CountryCode','AvgTone','NumArticles','SQLDATE', 'Actor1Geo_Lat', 'Actor1Geo_Long', 'ActionGeo_CountryCode', 'EventCode', 
                  'EventRootCode', 'Actor1Geo_CountryCode', 'Actor2Geo_CountryCode')\
        .filter("ActionGeo_CountryCode == 'SY'")\
        .filter("EventCode IN ('0233', '0234', '0256', '0333', '0334', '0356', '073', '074', '1033', '1034', '1056') OR EventRootCode IN ('18', '19', '20')" )\
        .withColumn('ImportanceTone', raw.AvgTone*raw.NumArticles)\
        .groupBy('Actor1CountryCode', 'SQLDATE', 'Actor1Geo_Lat', 'Actor1Geo_Long', 'EventRootCode', 'EventCode')\
        .sum('ImportanceTone', 'NumArticles')
        
data

AssertionError: col should be Column

In [5]:
raw.count()

8960662

In [6]:
data.repartition(1)\
    .write.option('header', 'true')\
    .csv('/home/javi/masterdatascience/TFM/data70')